In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
from pathlib import Path

import numpy

from atomlib import make
from atomlib.visualize import show_atoms_2d, show_atoms_3d
from atomlib.testing import assert_structure_equal, assert_files_equal

In [3]:
out_path = Path("").expanduser().resolve()
out_path.mkdir(exist_ok=True)

In [4]:
ceo2_conv = make.fluorite('CeO2', 5.47, cell='conv')
ceo2_prim = make.fluorite('CeO2', 5.47, cell='prim')
assert_structure_equal('CeO2_prim.xsf', ceo2_prim)

wobbles = {
    'Ce': 0.43 * 3/(8*numpy.pi**2),
    'O': 0.75 * 3/(8*numpy.pi**2),
}

ceo2_conv = ceo2_conv.with_wobble(wobbles)
ceo2_prim = ceo2_prim.with_wobble(wobbles)


print(ceo2_prim)

In [5]:
from atomlib.io import write_qe

write_qe(ceo2_conv, out_path / 'CeO2_pure.qe', {'Ce': 'Ce.GGA-PBE-paw-v1.0.UPF', 'O': 'O.pbe-n-kjpaw_psl.0.1.UPF'})

assert_files_equal('CeO2_pure.qe', out_path / 'CeO2_pure.qe')

In [10]:
ceo2_110_unit = make.slab(ceo2_prim, (1, 0, 0), (0, 1, 0))
ceo2_110 = ceo2_110_unit \
               .repeat_to_aspect(max_size=60.) \
               .repeat_to_z(200.)
print(f"cell size: {ceo2_110_unit.box_size} A")
print(f"supercell size: {ceo2_110.box_size} A")

ceo2_110_out = ceo2_110.round_near_zero()
ceo2_110.write(out_path / 'CeO2_110_pure.xsf')

assert_structure_equal('CeO2_110_pure_supercell.xsf', out_path / 'CeO2_110_pure.xsf')

show_atoms_3d(ceo2_110);

In [7]:
import polars

ceo2_110_gd = ceo2_110.with_symbol('Gd', ceo2_110.pos(1.0, 1.0, 0.0), frame='cell_frac')
ceo2_110_gd.write(out_path / 'CeO2_110_Gd_1.xsf')
assert_structure_equal('CeO2_110_Gd_1.xsf', out_path / 'CeO2_110_Gd_1.xsf')

assert len(ceo2_110_gd.filter(polars.col('elem') == 64)) == 1

ceo2_110_gd_all = ceo2_110.with_symbol('Gd', ceo2_110.pos(x=1.0, y=1.0), frame='cell_frac')
ceo2_110_gd_all.write(out_path / 'CeO2_110_Gd_all.xsf')
assert_structure_equal('CeO2_110_Gd_all.xsf', out_path / 'CeO2_110_Gd_all.xsf')

assert len(ceo2_110_gd_all.filter(polars.col('symbol') == 'Gd')) == 52

ceo2_110_v_ce = ceo2_110.filter(ceo2_110.pos(1.0, 1.0, 3.0).is_not(), frame='cell_frac')
ceo2_110_v_ce.write(out_path / 'CeO2_110_V_1.xsf')
assert_structure_equal('CeO2_110_V_1.xsf', out_path / 'CeO2_110_V_1.xsf')

assert len(ceo2_110_v_ce.filter(polars.col('elem') == 58)) == len(ceo2_110.filter(polars.col('elem') == 58)) - 1

In [7]:
scan_frac_110 = 2 / ceo2_110.cell.n_cells[:2]
scan_size_110 = ceo2_110.cell.box_size[:2] * scan_frac_110
scan_points_110 = numpy.ceil(4*scan_size_110*20e-3 / 0.0251).astype(int)

slice_thickness_110 = ceo2_110.cell.cell_size[2] / 2.

print(f"slice_thickness: {slice_thickness_110}")
print(f"scan_points: {scan_points_110}")
print(f"scan_step: {scan_size_110 / scan_points_110}")

zone = "110"
for (cell, name) in zip([ceo2_110, ceo2_110_gd, ceo2_110_gd_all, ceo2_110_v_ce],
                        ["pure", "Gd_Ce_1", "Gd_Ce_all", "V_Ce_1"]):
    cell.write(out_path / f"CeO2_{zone}_{name}.xsf")

    for (template, suffix) in zip(['template.mslice', 'template_tds.mslice', 'template_ptycho.mslice'],
                                ['', '_tds', '_tds_over20']):
        cell.write_mslice(
            out_path / f"CeO2_{zone}_{name}{suffix}.mslice", template=out_path / template,
            slice_thickness=slice_thickness_110,
            scan_points=scan_points_110,
            scan_extent=scan_frac_110
        )